In [6]:
import sys
import os

PROJECT_ROOT = '/home/astra/work/projects/lamedoc_new/project/'
sys.path.append(os.path.dirname(PROJECT_ROOT))
print(sys.path)

from src.utils.files_paths import init_paths

print(init_paths())

['', '/home/astra/anaconda3/envs/lamedoc/lib/python35.zip', '/home/astra/anaconda3/envs/lamedoc/lib/python3.5', '/home/astra/anaconda3/envs/lamedoc/lib/python3.5/plat-linux', '/home/astra/anaconda3/envs/lamedoc/lib/python3.5/lib-dynload', '/home/astra/anaconda3/envs/lamedoc/lib/python3.5/site-packages', '/home/astra/anaconda3/envs/lamedoc/lib/python3.5/site-packages/Sphinx-1.4.8-py3.5.egg', '/home/astra/anaconda3/envs/lamedoc/lib/python3.5/site-packages/setuptools-27.2.0-py3.5.egg', '/home/astra/anaconda3/envs/lamedoc/lib/python3.5/site-packages/IPython/extensions', '/home/astra/.ipython', '/home/astra/work/projects/lamedoc_new/project', '/home/astra/work/projects/lamedoc_new/project', '/home/astra/work/projects/lamedoc_new/project', '/home/astra/work/projects/lamedoc_new/project', '/home/astra/work/projects/lamedoc_new/project', '/home/astra/work/projects/lamedoc_new/project']
{'trigram_text_filepath': '/home/astra/db/processed_text/ngrams/wiki_trigram_25_12_2016_21_43.txt', 'bigram_t